In [1]:
from sklearn.metrics import matthews_corrcoef
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import numpy as np
import pickle
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
import h3
import folium



In [138]:
# PRETRAINED_MODEL_NAME = '/home/daril_kw/data/savings_for_60_rows/model_before_training_opti_full_for_para_60'
# TOKENIZER_DIR = '/home/daril_kw/data/savings_for_60_rows/tokenizer_final_opti_full_for_para_60'
# DATALOADER_DIR = "/home/daril_kw/data/savings_for_60_rows/test_dataloader_60.pt"


In [173]:
# This file test the first version of the model: classification with context

# PRETRAINED_MODEL_NAME = '/home/daril_kw/data/savings_for_60_rows/model_before_training_opti_full_for_para_60'
# TOKENIZER_DIR = '/home/daril_kw/data/savings_for_60_rows/tokenizer_final_opti_full_for_para_60'
# DATALOADER_DIR = "/home/daril_kw/data/savings_for_60_rows/test_dataloader_60.pt"


PRETRAINED_MODEL_NAME = '/home/daril_kw/data/model_saved_parallel_version_full_multinode'
TOKENIZER_DIR = '/home/daril_kw/data/savings_for_parallel_computing/tokenizer_final_opti_full'
DATALOADER_DIR = "/home/daril_kw/data/savings_for_parallel_computing/test_dataloader_full.pt"
DIR_TARGETS = "/home/daril_kw/data/savings_for_parallel_computing/targets_full_opti.pkl"


### Loading targets

In [3]:
with open(DIR_TARGETS, 'rb') as f:
    targets = pickle.load(f)

In [4]:



targets_dict={}
for i in range(len(targets)):
    if targets[i] not in targets_dict:
        targets_dict[targets[i]]=len(targets_dict)

targets_input=[targets_dict[targets[i]] for i in range(len(targets))]


In [103]:
targets_dict

{'8a39220f08a7fff': 0,
 '8a39220a9b4ffff': 1,
 '8a39220f060ffff': 2,
 '8a39220f3957fff': 3,
 '8a39220ad317fff': 4,
 '8a39220c4c17fff': 5,
 '8a39220f1497fff': 6,
 '8a39220f54a7fff': 7,
 '8a39220f1b0ffff': 8,
 '8a39220f310ffff': 9,
 '8a39220f1d07fff': 10,
 '8a39220f5407fff': 11,
 '8a39220c4d5ffff': 12,
 '8a39220f4347fff': 13,
 '8a39220f1d0ffff': 14,
 '8a39220f082ffff': 15,
 '8a39220f621ffff': 16,
 '8a392201a0effff': 17,
 '8a39220e271ffff': 18,
 '8a39220f4967fff': 19,
 '8a39220f54d7fff': 20,
 '8a39220e66d7fff': 21,
 '8a39220f1ceffff': 22,
 '8a39220e2657fff': 23,
 '8a392201bdb7fff': 24,
 '8a39220d016ffff': 25,
 '8a39220f035ffff': 26,
 '8a39220f572ffff': 27,
 '8a39220f00cffff': 28,
 '8a39220e24e7fff': 29,
 '8a39220f00effff': 30,
 '8a39220f5b2ffff': 31,
 '8a39220f2d6ffff': 32,
 '8a39220f57affff': 33,
 '8a39220d0a97fff': 34,
 '8a39220f1d6ffff': 35,
 '8a39220f1597fff': 36,
 '8a39220f622ffff': 37,
 '8a39220f4677fff': 38,
 '8a39220f0667fff': 39,
 '8a39220f401ffff': 40,
 '8a39220f1cf7fff': 41,
 '

In [5]:
# Construction de l'inverse du dictionnaire
targets_dict_inv = {v: k for k, v in targets_dict.items()}

In [105]:
targets_dict_inv

{0: '8a39220f08a7fff',
 1: '8a39220a9b4ffff',
 2: '8a39220f060ffff',
 3: '8a39220f3957fff',
 4: '8a39220ad317fff',
 5: '8a39220c4c17fff',
 6: '8a39220f1497fff',
 7: '8a39220f54a7fff',
 8: '8a39220f1b0ffff',
 9: '8a39220f310ffff',
 10: '8a39220f1d07fff',
 11: '8a39220f5407fff',
 12: '8a39220c4d5ffff',
 13: '8a39220f4347fff',
 14: '8a39220f1d0ffff',
 15: '8a39220f082ffff',
 16: '8a39220f621ffff',
 17: '8a392201a0effff',
 18: '8a39220e271ffff',
 19: '8a39220f4967fff',
 20: '8a39220f54d7fff',
 21: '8a39220e66d7fff',
 22: '8a39220f1ceffff',
 23: '8a39220e2657fff',
 24: '8a392201bdb7fff',
 25: '8a39220d016ffff',
 26: '8a39220f035ffff',
 27: '8a39220f572ffff',
 28: '8a39220f00cffff',
 29: '8a39220e24e7fff',
 30: '8a39220f00effff',
 31: '8a39220f5b2ffff',
 32: '8a39220f2d6ffff',
 33: '8a39220f57affff',
 34: '8a39220d0a97fff',
 35: '8a39220f1d6ffff',
 36: '8a39220f1597fff',
 37: '8a39220f622ffff',
 38: '8a39220f4677fff',
 39: '8a39220f0667fff',
 40: '8a39220f401ffff',
 41: '8a39220f1cf7fff',
 4

### Chargement du moelèle et des données

In [6]:

# Pour utiliser un GPU spécifique, utiliser cuda:1
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# load the prediction_dataloader
prediction_dataloader = torch.load(DATALOADER_DIR)
tokenizer = BertTokenizer.from_pretrained(TOKENIZER_DIR)


# On charge le modèle
model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME)
model.to(device)
print("we evaluate")
model.eval()

# On va stocker les prédictions et les vraies valeurs
predictions, true_labels, list_inputs_test = [], [], []


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/daril_kw/.venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


we evaluate


In [176]:
len(prediction_dataloader)

10418

### Exploration des données

In [177]:
# get the first batch in the prediction_dataloader

first_batch = next(iter(prediction_dataloader))
first_batch # this is a tuple of 3 elements: input_ids, attention_mask, labels
# first_batch[0].shape, first_batch[1].shape, first_batch[2].shape
first_batch = tuple(t.to(device) for t in first_batch) # we put the batch on the device


In [178]:
b_input_ids, b_input_mask, b_labels = first_batch


In [10]:

# move the batch to the device because we are using the GPU. the previous instruction tuple(t.to(device) for t in first_batch) is just a short cut for this
b_input_ids = b_input_ids.to(device)
b_input_mask = b_input_mask.to(device)
b_labels = b_labels.to(device)

### Préparation des données et séparation des trajectoires

In [12]:
def get_start_of_trajectory_based_on_proportion(input_ids, input_mask, proportion,context_length=6 ):
    """
    Cette fonction permet de recuperer le debut de la trajectoire en fonction de la proportion donnée et de la longueur du contexte
    La stratégie est la suivante:
    - On garde les tokens avant la proportion donnée et on remplace les tokens apres par des [PAD]: 0 mais tout en gardant ces tokens la car on va les utiliser plus tard pour tester la prediction

    Args:
        input_ids (torch.tensor): les ids des tokens
        input_mask (torch.tensor): le mask des tokens
        proportion (float): la proportion de la trajectoire que l'on veut garder: elle doit etre entre 0 et 1
        context_length (int): la longueur du contexte
        
    Returns:
        start_of_trajectory (torch.tensor): les ids des tokens de la trajectoire
        start_of_trajectory_mask (torch.tensor): le mask des tokens de la trajectoire
        tokens_to_predict list: les ids des tokens a predire: ce sont les tokens qui sont apres la proportion donnée qui sont remplacés par des [PAD]: 0
    """
    
    
    # On va verifier au la proportion est bien entre 0 et 1 si ce n'est pas le cas on va la mettre a 1 et retourner la trajectoire entiere sans token [SEP]
    if proportion < 0 or proportion > 1:
        proportion = 1
    
    # if proportion == 1:
        #On retire le token [SEP] à la fin de la trajectoire et ca sera ce token qui sera remplacé par des [PAD]: 0 et qui sera predit
    # La proportion a prendre est apres les tokens de contexte sachant que le format est le suivant: [CLS] contexte trajectoire [PAD]...
    # On detecte lepremier token [PAD] 
    first_pad = torch.where(input_ids == 0)[0]
    # la longueur est juste avant le premier token [PAD]
    length = first_pad[0] - context_length - 1 # on retire 1 pour le token [CLS] et 1 pour le token [SEP]
    # Calcul de la proportion de debut de trajectoire
    start_of_trajectory_length = int(length * proportion) # le resu
    # On met les tokens apres start_of_trajectory_length a 0 en les gardant pour la prediction
    tokens_to_predict = input_ids[start_of_trajectory_length + context_length :]
    # On retire les 0 de la prediction
    tokens_to_predict = tokens_to_predict[tokens_to_predict != 0]
    
    
    input_ids[start_of_trajectory_length + context_length :] = 0
    # On met le mask a 0 pour les tokens a predire
    input_mask[start_of_trajectory_length + context_length :] = 0
    # Maintenant, on doit ajouter le token [SEP] a la fin de la trajectoire et modifier le mask a la place du premier token [PAD]
    # On detecte lepremier token [PAD]
    first_pad = torch.where(input_ids == 0)[0]
    # On met le mask a 1 pour le premier token [PAD]
    input_mask[first_pad[0]] = 1
    # On ajoute le token [SEP] a la fin de la trajectoire
    input_ids[first_pad[0]] = 102
    
    
    return input_ids, input_mask, tokens_to_predict        

Test unitaire pour la construction du début de la trajectoire

In [20]:
trajectory_start, trajectory_start_mask,_ = get_start_of_trajectory_based_on_proportion(b_input_ids_clone[0], b_input_mask_clone[0], 1, context_length=6)
# pour tester,
all(trajectory_start * trajectory_start_mask == trajectory_start)

True

### Fonction d'ajout du token prédit

In [15]:
def add_prediction_to_input(input_ids, prediction, attention_masks, sep_token_id=102):
    """
    Cette fonction permet de remplacer le token de séparation par la prédiction et de déplacer le token de séparation à la position suivante
    
    Args:
    input_ids (torch.Tensor): les ids des tokens
    prediction (int): la prédiction
    attention_masks (torch.Tensor): le masque d'attention
    sep_token_id (int): l'id du token de séparation par défaut 102 mais cela peut changer
    
    Returns:
    input_ids (torch.Tensor): les ids des tokens avec la prédiction ajoutée et le token de séparation déplacé
    """
    
    #On cherche la position du token de séparation
    sep_token_position = (input_ids == sep_token_id).nonzero(as_tuple=True)[0]
    # On remplace le token de séparation par la prédiction
    input_ids[sep_token_position] = prediction
    # On déplace le token de séparation à la position suivante
    input_ids[sep_token_position+1] = sep_token_id

    # Mise à jour du masque d'attention
    attention_masks[sep_token_position+1] = 1 # Dans le masque d'attention, on met à 1 tous les tokens n´étant pas des pads
    return input_ids , attention_masks

In [125]:
def auto_regressive_prediction(model, start_of_the_trajectory_ids, start_of_the_trajectory_mask, tokenizer, limit_number_of_prediction = 512,):
    
    """
    Cette fonction permet de predire les tokens de la trajectoire en utilisant un modele de prediction
    Et se limite a un nombre de tokens predits ou a la prediction de du token [SEP]
    
    Args:
    model (torch.nn.Module): le modele de prediction
    start_of_the_trajectory_ids (torch.Tensor): les ids des tokens de la trajectoire
    start_of_the_trajectory_mask (torch.Tensor): le masque des tokens de la trajectoire
    limit_number_of_prediction (int): le nombre de tokens a predire
    
    Returns:
    predictions (list):  les ids des tokens predits
    predicted_h3_tokens (list): les tokens h3 predits
    predicted_trajectory (list): les ids des tokens de la trajectoire predite
    """
    
    # On met le modele en mode evaluation
    with torch.no_grad():
        # On initialise les predictions
        predictions = []
        
        predicted_h3_tokens = []
       
        # On initialise le nombre de tokens predits
        number_of_predictions = 0
        # On initialise le token de prediction
        predicted_token = 0
        # On fait une boucle pour predire les tokens
        while predicted_token != 102 and number_of_predictions < limit_number_of_prediction:
            
            # On cree un batch avec les ids et le masque.la methode unsqueeze_mask permet d'ajouter une dimension pour avoir un batch de taille 1
            unsqueeze_ids = start_of_the_trajectory_ids.unsqueeze(0) # On ajoute une dimension pour avoir un batch de taille 1
            unsqueeze_mask = start_of_the_trajectory_mask.unsqueeze(0) # On ajoute une dimension pour avoir un batch de taille 1
            
            # On predit le token suivant
            outputs = model(input_ids=unsqueeze_ids, attention_mask=unsqueeze_mask)
            # On recupere la prediction
            logits = outputs[0]
            # On recupere le token predit ayant la plus grande probabilité
            best_class = torch.argmax(logits, dim=1) 
            h3_token = targets_dict_inv[best_class.item()]
            # On ajoute le token h3 a la liste des tokens predits
            predicted_h3_tokens.append(h3_token)
            # On encode le token
            predicted_token = tokenizer.encode(h3_token, add_special_tokens=False)[0]
            # On ajoute le token predit a la liste des predictions
            predictions.append(predicted_token)
            # On ajoute le token predit a la trajectoire
            start_of_the_trajectory_ids, start_of_the_trajectory_mask = add_prediction_to_input(start_of_the_trajectory_ids, predicted_token, start_of_the_trajectory_mask)
            # On incremente le nombre de predictions
            number_of_predictions += 1
    return  predictions,predicted_h3_tokens, start_of_the_trajectory_ids

Test de la fonction

In [ ]:
# the size of b_input_ids is 
b_input_ids.size()

torch.Size([12, 512])

## Show H3 trajectory on the map

In [55]:
def get_h3_token_without_context(input_ids, tokenizer, context_token_number=6):
    """
   
    Cette fonction prend les ids des tokens, le tokenizer et le nombre de tokens de contexte,
    Commence à vérifier les tokens après le nombre de tokens de contexte +1 
    token CLS s'il y a des tokens h3 et les renvoie
    Ceci sachant que les inputs sont sous la forme: [CLS] jetons_de_context jetons_h3 [SEP]
    
    Args:
    input_ids (torch.Tensor): les ids des tokens
    tokenizer (transformers.tokenizer): le tokenizer
    context_token_number (int): le nombre de tokens de contexte
    
    Returns:
    list: une liste des tokens h3
    
    """
    # On retire tous les ids des tokens qui sont des [PAD]
    inputs_ids = input_ids[input_ids != 0]
    
    h3_tokens = []
    detokenized_tokens = tokenizer.decode(input_ids)
    tokens = detokenized_tokens.split()
    for i in range(context_token_number+1, len(tokens)):
        if h3.h3_is_valid(tokens[i]):
            h3_tokens.append(tokens[i])
    return h3_tokens

In [158]:
def add_h3_tokens_on_map(h3_tokens=[],add_centers=True, polyline_color="red", polygon_color="green", remove_duplicates=True,show_start_marker=True, show_end_marker=True, map=m):
    """
     Cette fonction permet d'ajouter des h3 tokens sur une carte folium
     
        Args:
        h3_tokens (list): les h3 tokens
        add_centers (bool): si True, on ajoute les centres des h3 tokens
        polyline_color (str): la couleur de la polyline
        polygon_color (str): la couleur du polygone
        remove_duplicates (bool): si True, on retire les doublons
        show_start_marker (bool): si True, on affiche le point de depart
        show_end_marker (bool): si True, on affiche le point d'arrivee
        map (folium.Map): la carte folium
        
        Returns:
        folium.Map: la carte folium
    """
    
    if remove_duplicates:
        # On retire les doublons et on garde l'ordre
        h3_tokens = list(dict.fromkeys(h3_tokens))
  
    h3_boundaries = [h3.h3_to_geo_boundary(h3_token) for h3_token in h3_tokens]

    polyline = []
    if add_centers:
        polyline = [h3.h3_to_geo(h3_token) for h3_token in h3_tokens]
        folium.PolyLine(
            locations=polyline,
            color=polyline_color,
            opacity=1,
            weight=2.5,
        ).add_to(map)
    
    for i in range(len(h3_boundaries)):
        folium.Polygon(
            locations=h3_boundaries[i],
            color=polygon_color,
            fill=True,
            fill_color=polygon_color,
            fill_opacity=0.4,
        ).add_to(map)
    # Maintenant, nous allons marquer point de debut et le point de fin
    if len(polyline) > 0:
        if show_start_marker:
            folium.Marker(
                location=polyline[0],
                popup="Start",
                icon=folium.Icon(color="green"),
            ).add_to(map)
        if show_end_marker:
            folium.Marker(
                location=polyline[-1],
                popup="End",
                icon=folium.Icon(color="red"),
            ).add_to(map)
    return m
       

### Fonction gloale de prédiction et d´affichage sur la map

In [161]:
def predict_autoregressively_and_show_on_the_map(input_ids, input_mask, model, tokenizer, context_length=6, proportion=0.8, limit_number_of_prediction=30):
    """
    Cette fonction permet de predire les tokens de la trajectoire en utilisant un modele de prediction et de les afficher sur la carte
    
    Args:
    input_ids (torch.Tensor): les ids des tokens
    input_mask (torch.Tensor): le mask des tokens
    model (torch.nn.Module): le modele de prediction
    tokenizer (transformers.tokenizer): le tokenizer
    context_length (int): la longueur du contexte
    proportion (float): la proportion de la trajectoire que l'on veut garder
    limit_number_of_prediction (int): le nombre de tokens a predire
    
    """
    trajectory_start, trajectory_start_mask,_ = get_start_of_trajectory_based_on_proportion(input_ids, input_mask, proportion, context_length)
    predicted_part_ids, predicted_h3_tokens ,predicted_trajectory = auto_regressive_prediction(model, trajectory_start, trajectory_start_mask, tokenizer, limit_number_of_prediction = limit_number_of_prediction)
    trajectory_start_h3_tokens = get_h3_token_without_context(trajectory_start, tokenizer)
    fist_point = list(h3.h3_to_geo(trajectory_start_h3_tokens[0]))
    map = folium.Map(location=fist_point, zoom_start=10)
    add_h3_tokens_on_map(trajectory_start_h3_tokens, add_centers=True, polyline_color="blue", polygon_color="red", remove_duplicates=False, show_start_marker=True, show_end_marker=False,map=map) 
    # Ajout des tokens predits
    add_h3_tokens_on_map(predicted_h3_tokens, add_centers=True, polyline_color="red", polygon_color="purple", remove_duplicates=False, show_start_marker=False, show_end_marker=True, map=map)
    return map

test de la fonction globale de prédiction et d´affichage sur la map

In [ ]:
predict_autoregressively_and_show_on_the_map(b_input_ids_clone.clone()[11], b_input_mask_clone.clone()[11], model, tokenizer, context_length=6, proportion=0.2, limit_number_of_prediction=30)